In [15]:
import random 
import csv

In [16]:
CROSSOVER_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.05
CARRY_PERCENTAGE = 0.2
POPULATION_SIZE = 100
MULTI_START_COUNT = 10

<h1>Part 0 : Read inputs from CSV file</h1>
<p>it is look very simple. we just can use <b style="color:blue">csv library</b> and its methods to extract first row of file from hole rows and save the others in list namsd snacks</p>

In [17]:
snacks = []
with open("snacks.csv" , 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row in csvreader:
        snacks.append(row)
file.close()
print(snacks)
len(snacks)


[['MazMaz', '10', '10'], ['Doogh-e-Abali', '15', '10'], ['Nani', '5', '5'], ['Jooj', '7', '15'], ['Hot-Dog', '20', '15'], ['Chips', '8', '6'], ['Nooshaba', '12', '8'], ['Shokolat', '6', '7'], ['Chocoroll', '9', '12'], ['Cookies', '11', '11'], ['Abnabat', '4', '4'], ['Adams-Khersi', '14', '9'], ['Popcorn', '16', '13'], ['Pastil', '3', '7'], ['Tordilla', '10', '9'], ['Masghati', '5', '6'], ['Ghottab', '7', '10'], ['Saghe-Talaei', '9', '11'], ['Choob-Shoor', '13', '12']]


19

<h1>Part1: Basic Concepts</h1>
<p>Each gene will be considered as a number between 0 and 1. n-th gene correspond that we peak what ratio of n-th food and finaly Each chromosome will consist of some genes and the number of these genes will be same as number of foods or exactly number of rows in csv file<i>(exept first row)</i>.</p>

<h1>Part2: initial Population</h1>
<p>the <b>POPULATION_SIZE</b> variable defined to set the population size.</p>

<h1>Part3: Fitness Function</h1>
<p>there are 3 parameters in <b style="color:green">Fractional Knapsack</b> problem that we have them as input of our issue:</p>
<ul type="square">
    <li>MIN_VALUE</li>
    <li>MAX_WEIGHT</li>
    <li>SNACK_NUMBER_INTERVAL --> <i style="color:gray">(SNI an abbreviation)</i></li>
</ul>

<p>for the first item, if value of chromosome was bigger equal than min_value , it gets 1 point otherwise 0.</p>
<p>for the second item, if weight of chromosome was smaller equal than max_weight , it gets 1 point otherwise 0.</p>
<p>finaly for third and last item, we extract the posetive and non-zero elements of each chromosome which shows how many snacks we peak.if this value was not in SNI , that chromosome gives 0 point otherwhise gives 1 point.</p>

<b>at last we define the <span style="color:blue">Fitness Function</span> as bellow:</b>

$fitness = \sum{C_i} $

<p>C1 = chromose.value >= minValue ? 1 : 0<p>
<p>C2 = chromose.weight <= maxWeight ? 1 : 0<p>
<p>C3 = chromose.nonzeroElementsNumber in SNI ? 1 : 0<p>

<p>with this implementation , all of our soloution chromosomes will have fitnees(chromosome) <span>=</span> 3</p>

<h1>Part4: implementing CrossOver and Mutation Function and create next Generation</h1>
<h2 style="color:blue">CrossOver</h2>
<p>for CrossOver function , first we generate a random number between 0 and 1. Then, for each pair of chromosomes in mating pool, if the number is more than CROSSOVER_PROBABILITY, we select the chromosomes for the next phase without any change. Otherwise, we swap the genes in the middle of two random points, between the chromosomes. In this case, we have created a pair of child chromosomes from their parents.</p>
<h2 style="color:blue">Mutation</h2>
<p>for Mutaion function , For each gene in each chromosome of the crossover pool, we generate a random number between 0 and 1. If the number is less than MUTATION_PROBABILITY, we change the gene to a random number between 0 and 1 otherwise the gene saves its previous value.</p>

<h1>Part5: Running algorithm</h1>
<p>here is complete code for algorithm:</p>

In [18]:
class Knapsack:
    snacks_number : int
    max_weight : int
    gen_count : int
    min_value : int
    interval_begin : int
    interval_end : int
    population : list[list]

    def __init__(self,snacks_number,max_weight,min_value,gen_count ,interval_begin , interval_end):
        self.max_weight = max_weight
        self.min_value = min_value
        self.snacks_number = snacks_number
        self.interval_begin = interval_begin
        self.interval_end = interval_end
        self.gen_count = gen_count
    
    def makeFirstPopulation(self) -> list[list]:
        population = []
        for _ in range(POPULATION_SIZE):
            chromosome = [0] * self.snacks_number
            randomInt = random.randint(self.interval_begin , self.interval_end)
            for j in range(randomInt):
                chromosome[j] = random.randint(0,100)/100
            for j in range(randomInt , self.snacks_number - randomInt):
                chromosome[j] = 0
            random.shuffle(chromosome)
            population.append(chromosome)
        return population
    
    def find_solution(self , multi_start_count:int = 1)-> tuple[list,bool]:
        bestsolution = None
        for _ in range (multi_start_count):
            self.population = self.makeFirstPopulation()
            for _ in range (self.gen_count):
                random.shuffle(self.population)

                fitness = [self.calculate_fitness(self.population[i])for i in range(POPULATION_SIZE)]
                if max(fitness)==3:
                    return self.population[fitness.index(3)] , True
                
                bestchromosomes = [x for _ , x in sorted(zip(fitness,self.population),key=lambda pair : pair[0] , reverse= True)] 
                if bestsolution is None or self.calculate_fitness(bestsolution) < self.calculate_fitness(bestchromosomes[0]):
                    bestsolution = bestchromosomes[0]
                carriedChromosomes = []
                for i in range(0,int(POPULATION_SIZE*CARRY_PERCENTAGE)):
                    carriedChromosomes.append(bestchromosomes[i])

                matingpool = self.create_mating_pool(bestchromosomes)
                crossoverpool = self.create_crossover_pool(matingpool)
                self.population.clear()

                for i in range (POPULATION_SIZE - int(POPULATION_SIZE*CARRY_PERCENTAGE)):
                    self.population.append(self.mutation(crossoverpool[i]))

                self.population.extend(carriedChromosomes)
        return bestsolution , False
     
    
    def create_mating_pool(self,bestchromosomes:list[list]) -> list[list]:
        ranks = list(reversed(range(1 , POPULATION_SIZE + 1)))
        matingpool = []
        for i in range (POPULATION_SIZE):
            for _ in range(ranks[i]):
                matingpool.append(bestchromosomes[i])
        random.shuffle(matingpool)
        return matingpool[:POPULATION_SIZE]
    
    
    def create_crossover_pool(self, matingpool:list[list]) -> list[list]:
        crossoverpool = []
        for i in range (0 , len(matingpool)-1 , 2):
            if random.random() > CROSSOVER_PROBABILITY:
                crossoverpool.append(matingpool[i])
                crossoverpool.append(matingpool[i + 1])
            else:
                child = self.crossover(matingpool[i],matingpool[i + 1])
                crossoverpool.extend(child)
        return crossoverpool
    
    def crossover(self, chromosome1:list,chromosome2:list) -> tuple[list,list]:
        breakpoint_one = random.randint(0,self.snacks_number - 1)
        breakpoint_two = random.randint(0,self.snacks_number - 1)
        if breakpoint_one > breakpoint_two:
            breakpoint_one , breakpoint_two = breakpoint_two , breakpoint_one
        
        chromosome1 = list(chromosome1)
        chromosome2 = list(chromosome2)
        for i in range (breakpoint_one,breakpoint_two):
            chromosome1[i] , chromosome2[i] = chromosome2[i] , chromosome1[i]
        return chromosome1 , chromosome2
    
    def mutation(self,chromosome:list) -> list:
        chromosome = list(chromosome)
        for i in range(len(chromosome)):
            if random.random() < MUTATION_PROBABILITY:
                chromosome[i] = random.randint(0,100)/100
        return chromosome

    def calculate_fitness(self,chromosome : list) -> float:
        chromosome_weight = sum(map(lambda x : x[0]*x[1] , zip([eval(snack[1])for snack in snacks], chromosome)))
        chromosome_value = sum(map(lambda x : x[0]*x[1] , zip([eval(snack[2])for snack in snacks], chromosome)))
        peaked_item_number = sum(1 for gene in chromosome if gene != 0)
        return (self.max_weight >= chromosome_weight ) + (chromosome_value  >= self.min_value) + (self.interval_begin <= peaked_item_number <= self.interval_end)
        
       

    


<h1>Part6: Questions</h1>
<div style="border:2px solid gray">
<h1>1.What problems does a small or large primary population cause?</h1>
<p>If the population size is very small, the algorithm may not be able to find the best solution because it may not have enough chromosomes to select from. On the other hand, if the population size is very large, the algorithm may take a long time to find the best solution and it may be unnecessary to spend that much time.</p>
</div>
<div style="border:2px solid gray">
<h1>2.What happens if the population size increases in each generation?</h1>
<p>This may return better solution but increases the time complexity and the memory. Also, it is unnecessary to do this because we can remove the chromosomes with the lowest fitness from the population in order to keep the population size constant.</p></div>
<div style="border:2px solid gray">
<h1>3.What is the effects of crossover and mutation? Is it possible to use only one of them?</h1>
<p>Crossover is used to create new chromosomes from the existing chromosomes. Mutation is used to change the genes of the chromosomes. For examp if we use only crossover, the neightbor genes may be stay same all time. in the other side ,If we use only mutation, it can be very random to change each gene. so if we use only one of these two , we may not be able to find the best solution. It is important to note that the crossover and mutation probabilities should be chosen carefully. The crossover probability is usually at least 80% and the mutation probability is usually at most 5%.</p></div>
<div style="border:2px solid gray">
<h1>4.how we can reach to solution faster in this special problem?</h1>
<p>Fitness function, parameters such as the probability of crossover and mutation, and the population size should be chosen carefully. Also, the crossover and mutation functions can affect the performance of the algorithm.</p></div>
<div style="border:2px solid gray">
<h1>5.why chromosomes don't change after a while , and how we can solve it?</h1>
<p>A common problem in genetic algorithms is that it may stop at a local maximum instead of the global maximum. Mutation is a good way to solve this problem. Also, we can limit the number of generations in order to stop the algorithm if it is not converging. In this case, we may also use multi-start to increase the probability of finding the global maximum.</p></div>
<div style="border:2px solid gray">
<h1>6.how we can terminate algorithm if it doesn't return an appropriate solution?</h1>
<p>As mentioned in the previous question, we can limit the number of generations in order to stop the algorithm if there exists no solution. We can use the following formula for the generation limit:</p><div style="color:pink ;">GenLimit = 2 * POPULATION_SIZE * Number_of_Snacks</div></div>

<strong>Now we distinguish our parameter for problem:</strong>

In [19]:
MAX_W = 10
MIN_V = 12
Int_begin = 2
Int_end = 4
GenCount = POPULATION_SIZE * 2 * len(snacks)

<p>create instance using constructor and pass the parameter to it:</p>

In [20]:
knapsack = Knapsack(len(snacks),MAX_W,MIN_V,GenCount,Int_begin,Int_end)
solution , isFind = knapsack.find_solution(1)
if not isFind:
    print("No true solution found but the best solution is: ")
print(*solution)



0.08 0 0 0 0 0 0 0.12 0 0 0 0 0 0.77 0 0.86 0 0 0


<p>at last we just should print the result as we want:<p>

In [21]:
indexes = [solution.index(ratio) for ratio in solution if ratio != 0]
Total_Weight = 0
Total_Value = 0

for index in indexes:
    Total_Value += solution[index] * eval(snacks[index][2])
    Total_Weight += solution[index] * eval(snacks[index][1])

for index in indexes:
    print(snacks[index][0] + " : " + str(solution[index] * eval(snacks[index][1])))

print("Total Weight : " + str(Total_Weight))
print("Total Value : " + str(Total_Value))



MazMaz : 0.8
Shokolat : 0.72
Pastil : 2.31
Masghati : 4.3
Total Weight : 8.129999999999999
Total Value : 12.190000000000001
